In [1]:
import gc
import os

import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
tf.config.list_physical_devices('GPU')

2023-12-17 13:48:17.722023: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-17 13:48:17.746076: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-17 13:48:17.746204: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_dir = 'imgs/train'
test_dir = 'imgs/test'
csv_file = 'driver_imgs_list.csv'

In [4]:
def load_images_from_folder(base_folder, size=(112, 112)):
    total_images = sum([len(files) for r, d, files in os.walk(base_folder)])

    all_images = np.empty((total_images, size[0], size[1], 3), dtype=np.uint8)
    all_labels = np.empty(total_images, dtype=np.uint8)

    idx = 0
    for folder in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder)
        
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                
                try:
                    img = Image.open(img_path).resize(size)
                    all_images[idx] = np.array(img)
                    all_labels[idx] = folder[-1]
                    idx += 1
                except IOError:
                    print(f"Error in reading {img_path}. Skipping.")

    return all_images, all_labels

In [5]:
base_folder = 'imgs/train'
images, labels = load_images_from_folder(base_folder)

In [6]:
X = images.astype('float32') / 255

In [7]:
Y = to_categorical(labels, 10)

In [8]:
n_folds = 10

kfold = KFold(n_folds, shuffle=True, random_state=42)

histories = []
accuracies = []
precisions = []
recalls = []
f1_scores = []
conf_matrices = []

In [13]:
i = 0
for train, test in kfold.split(X, Y):
    print(i)
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=10)
    histories.append(history)

    y_pred = model.predict(X[test])
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(Y[test], axis=1)

    accuracies.append(accuracy_score(y_true, y_pred_classes))
    precisions.append(precision_score(y_true, y_pred_classes, average='weighted'))
    recalls.append(recall_score(y_true, y_pred_classes, average='weighted'))
    f1_scores.append(f1_score(y_true, y_pred_classes, average='weighted'))
    conf_matrices.append(confusion_matrix(y_true, y_pred_classes))
    i += 1
    
    del model
    gc.collect()

0
Epoch 1/10
631/631 [==============================] - 4s 5ms/step - loss: 0.4577 - accuracy: 0.8798 - val_loss: 0.0437 - val_accuracy: 0.9893
Epoch 2/10
631/631 [==============================] - 3s 5ms/step - loss: 0.0173 - accuracy: 0.9967 - val_loss: 0.0208 - val_accuracy: 0.9929
Epoch 3/10
631/631 [==============================] - 3s 5ms/step - loss: 0.0038 - accuracy: 0.9996 - val_loss: 0.0218 - val_accuracy: 0.9938
Epoch 4/10
631/631 [==============================] - 3s 5ms/step - loss: 0.0026 - accuracy: 0.9997 - val_loss: 0.0269 - val_accuracy: 0.9911
Epoch 5/10
631/631 [==============================] - 3s 5ms/step - loss: 0.0157 - accuracy: 0.9951 - val_loss: 0.0268 - val_accuracy: 0.9897
Epoch 6/10
631/631 [==============================] - 3s 5ms/step - loss: 0.0031 - accuracy: 0.9993 - val_loss: 0.0072 - val_accuracy: 0.9982
Epoch 7/10
631/631 [==============================] - 3s 5ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.0128 - val_accuracy: 0.9951
Epoc

In [15]:
np.mean(accuracies)

0.9965438714478149

In [16]:
np.mean(precisions)

0.9965585423975574

In [17]:
np.mean(recalls)

0.9965438714478149

In [18]:
np.mean(f1_scores)

0.9965424989057734

In [19]:
conf_matrices

[array([[257,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0, 232,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 279,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0, 228,   0,   0,   1,   0,   0,   0],
        [  0,   0,   0,   1, 219,   0,   0,   0,   0,   0],
        [  2,   0,   0,   0,   1, 244,   1,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0, 216,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 180,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 169,   0],
        [  0,   0,   0,   0,   0,   0,   1,   0,   2, 210]]),
 array([[246,   0,   0,   0,   0,   1,   0,   0,   0,   0],
        [  0, 253,   0,   0,   0,   0,   0,   0,   0,   1],
        [  0,   0, 233,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   1, 250,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 244,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 214,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   1,

In [25]:
np.sum(conf_matrices, axis=0)

array([[4962,    2,    2,    2,    5,    2,    0,    1,    0,    2],
       [   1, 4528,    0,    2,    0,    0,    0,    0,    0,    3],
       [   2,    1, 4622,    0,    0,    0,    4,    1,    1,    3],
       [   3,    2,    2, 4679,    3,    1,    2,    0,    0,    0],
       [   1,    0,    0,    5, 4644,    0,    0,    0,    2,    0],
       [  13,    0,    0,    0,    6, 4597,    2,    3,    1,    2],
       [   2,    0,    1,    1,    0,    2, 4642,    2,    0,    0],
       [   2,    0,    0,    0,    0,    0,    1, 4000,    1,    0],
       [   2,    0,    1,    1,    0,    1,    4,    2, 3794,   17],
       [   1,    1,    0,    0,    0,    4,    4,    2,   21, 4225]])